In [226]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)

In [227]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')
df=df.dropna(axis=0)

In [228]:
y=df['EColi.']
X=df.loc[:, ['陰影起伏', '曲率', '傾斜角', '傾斜方向','高度', '近くの川への距離', '鉄道ポイントへの距離', '主要道路への距離', 
             'near_distance_syorizyou', 'supply_hours', 'no water days', 'total population ', 'population served', 'popu-served', 
             'number taps', 'pipes(km)', 'pipes/tabs \n個/km', 'served/pipes', '(popu-served)/pipes\n  /km', 
             'Age of oldeset\npipe ', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [229]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [230]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [231]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [232]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

In [233]:
X_train_std

,陰影起伏,曲率,傾斜角,傾斜方向,高度,近くの川への距離,鉄道ポイントへの距離,主要道路への距離,near_distance_syorizyou,supply_hours,no water days,total population,population served,popu-served,number taps,pipes(km),pipes/tabs \n個/km,served/pipes,(popu-served)/pipes\n /km,Age of oldeset\npipe,ST,RSF,FL,PF,RF
218,-0.686850,-0.382187,0.048990,-0.410915,-1.159545,-0.327503,-1.265976,-0.135508,-0.070461,0.939600,-1.478259,-1.032852,-1.008378,-1.129822,-1.022905,-1.036154,-0.031214,-1.256239,-0.508817,-0.784953,1,1,0,0,0
100,1.831442,-1.612959,0.077027,1.317255,0.879267,0.004160,0.949064,-0.448003,-0.178807,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
147,-0.686850,0.110122,0.083950,0.323822,0.939576,0.527216,1.045501,-0.147035,-0.094643,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
190,-0.327094,0.725508,0.080769,0.610908,-1.188024,-0.724693,-0.980053,-0.163948,-0.394109,0.380087,-0.877575,-1.024947,-1.014043,-1.062279,-1.033523,-1.070890,-0.023260,-1.165195,0.956916,-1.091538,0,0,0,0,0
192,-0.226362,-0.012955,0.082693,-1.455383,-1.146142,-0.883740,-1.183121,1.309977,-0.016418,-0.515133,0.040044,-0.925869,-0.999170,-0.593647,-1.008196,-1.059415,-0.179687,-0.272058,5.788956,-1.295928,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,-0.686850,-0.751419,0.072918,0.057667,-0.975264,-0.532585,-0.258298,0.944393,0.031416,1.611015,-1.406907,-0.858066,-0.925323,-0.553148,-0.861831,-0.523634,-0.042145,-1.324718,-0.331289,-0.963794,0,0,0,0,0
156,-0.686850,1.463971,0.085376,-0.390698,0.931200,-0.800241,1.049782,-0.202599,0.026095,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
16,-0.686850,0.233199,0.080769,-1.263421,-1.132740,-0.364624,-1.329180,-0.221402,-0.399950,0.044380,-0.334968,-1.026417,-1.014292,-1.069108,-1.031457,-1.066652,-0.043024,-1.269909,0.629604,-0.963794,0,0,0,0,0
144,-0.686850,-0.136033,0.084819,0.003578,1.019990,0.350658,1.092903,-0.304388,0.119831,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1


# SVC

In [343]:
from sklearn.svm import SVC
svm=SVC(kernel='linear', C=100, random_state=1)

In [344]:
svm.fit(X_train_std, y_train)

SVC(C=100, kernel='linear', random_state=1)

In [345]:
svm.score(X_train_std, y_train)

0.7171052631578947

In [346]:
svm.score(X_test_std, y_test)

0.6666666666666666

In [338]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
pipe_svc=make_pipeline( SVC(random_state=1))

param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid=[{'svc__C':param_range, 'svc__kernel':['linear']},
            {'svc__C':param_range}
           ]

In [339]:
gs=GridSearchCV(estimator=pipe_svc,
                param_grid=param_grid,
                scoring='accuracy',
                cv=10,
                n_jobs=-1)

In [340]:
gs=gs.fit(X_train_std, y_train)

In [341]:
gs.best_score_

0.6195833333333333

In [342]:
gs.best_params_

{'svc__C': 100.0, 'svc__kernel': 'linear'}

# ロジスティック回帰

In [243]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=10)
grid_search.fit(X_train_std, y_train)

print("Best parameters : {}".format(grid_search.best_params_))
print("Best cross-validation score : {:.3f}".format(grid_search.best_score_))

C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

Best parameters : {'C': 10}
Best cross-validation score : 0.600


C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

In [244]:
lr=LogisticRegression(C=10, random_state=1)

In [245]:
lr.fit(X_train_std, y_train)

C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10, random_state=1)

In [246]:
lr.score(X_train_std, y_train)

0.7171052631578947

In [247]:
lr.score(X_test_std, y_test)

0.6515151515151515

# random forest

In [248]:
from sklearn.ensemble import RandomForestClassifier

In [285]:
forest=RandomForestClassifier(criterion='entropy', n_estimators=17, random_state=1)

In [324]:
forest.fit(X_train_std, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=17, random_state=1)

In [325]:
forest.score(X_train_std, y_train)

0.993421052631579

In [326]:
forest.score(X_test_std, y_test)

0.7727272727272727

In [284]:
%%time
from tqdm import tqdm
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


#条件設定
max_score = 0
SearchMethod = 0
RFC_grid = {RandomForestClassifier(): {"n_estimators": [i for i in range(1, 100)],
                                       "criterion": ["gini", "entropy"],
                                       }}

#ランダムフォレストの実行
for model, param in tqdm(RFC_grid.items()):
    clf = GridSearchCV(model, param, cv=10)
    clf.fit(X_train_std, y_train)
    pred_y = clf.predict(X_test_std)
    score = f1_score(y_test, pred_y, average="micro")

    if max_score < score:
        max_score = score
        best_param = clf.best_params_
        best_model = model.__class__.__name__

print("ベストスコア:{}".format(max_score))
print("モデル:{}".format(best_model))
print("パラメーター:{}".format(best_param))

#ハイパーパラメータを調整しない場合との比較
model = RandomForestClassifier()
model.fit(X_train_std, y_train)
score = model.score(X_test_std, y_test)
print("")
print("デフォルトスコア:", score)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [05:38<00:00, 338.83s/it]


ベストスコア:0.6666666666666666
モデル:RandomForestClassifier
パラメーター:{'criterion': 'entropy', 'n_estimators': 17}

デフォルトスコア: 0.7121212121212122
Wall time: 5min 39s


In [361]:
%%time
from tqdm import tqdm
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


#条件設定
max_score = 0
SearchMethod = 0
RFC_grid = {RandomForestClassifier(): {"n_estimators": [i for i in range(1, 100)],
                                       "criterion": ["gini", "entropy"],
                                       }}

#ランダムフォレストの実行
for model, param in tqdm(RFC_grid.items()):
    clf = GridSearchCV(model, param, cv=10)
    clf.fit(X_train_std, y_train)
    pred_y = clf.predict(X_test_std)
    score = clf.score(X_test_std, y_test)

    if max_score < score:
        max_score = score
        best_param = clf.best_params_
        best_model = model.__class__.__name__

print("ベストスコア:{}".format(max_score))
print("モデル:{}".format(best_model))
print("パラメーター:{}".format(best_param))

#ハイパーパラメータを調整しない場合との比較
model = RandomForestClassifier()
model.fit(X_train_std, y_train)
score = model.score(X_test_std, y_test)
print("")
print("デフォルトスコア:", score)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [03:47<00:00, 227.07s/it]


ベストスコア:0.6818181818181818
モデル:RandomForestClassifier
パラメーター:{'criterion': 'gini', 'n_estimators': 37}

デフォルトスコア: 0.6818181818181818
Wall time: 3min 47s


In [362]:
forest=RandomForestClassifier(criterion='gini', n_estimators=37, random_state=1)

In [363]:
forest.fit(X_train_std, y_train)

RandomForestClassifier(n_estimators=37, random_state=1)

In [364]:
forest.score(X_test_std, y_test)

0.6666666666666666

In [365]:
forest.score(X_train_std, y_train)

1.0

In [331]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(estimator=forest,
                      X=X_train_std, y=y_train,
                      cv=15, n_jobs=1)

In [332]:
score

array([0.45454545, 0.63636364, 0.5       , 0.4       , 0.6       ,
       0.6       , 0.6       , 0.5       , 0.8       , 0.6       ,
       0.6       , 0.3       , 0.6       , 0.6       , 0.7       ])

In [333]:
score.mean()

0.5660606060606059

# Boosted forest

In [253]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [357]:
tree=DecisionTreeClassifier(criterion='entropy', max_depth=7, random_state=1)
ada=AdaBoostClassifier(base_estimator=tree, n_estimators=96, learning_rate=1, random_state=1)

In [358]:
ada.fit(X_train_std, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=7,
                                                         random_state=1),
                   learning_rate=1, n_estimators=96, random_state=1)

In [359]:
ada.score(X_train_std, y_train)

1.0

In [360]:
ada.score(X_test_std, y_test)

0.5303030303030303

In [352]:
from sklearn.datasets import load_digits
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

def main():
    digits = load_digits()
    ada = AdaBoostClassifier(n_estimators=100) # ついでに増やす
    params = {"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
             
    }
    cv = GridSearchCV(ada, params, cv=10, n_jobs=-1, verbose=2)

    cv.fit(X_train_std, y_train)
    
    print(cv.best_params_)
    pred = cv.predict(X_test_std)
    print(classification_report(y_test, pred))

if __name__ == "__main__":
    main()

Fitting 10 folds for each of 1485 candidates, totalling 14850 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 450 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 653 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1609 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2188 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 2889 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 3792 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 4777 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 5854 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 7366 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 8596 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 10528 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 13432 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 14850 out of 148

{'base_estimator': DecisionTreeClassifier(max_depth=7), 'learning_rate': 1.0, 'n_estimators': 96}
              precision    recall  f1-score   support

           0       0.57      0.57      0.57        30
           1       0.64      0.64      0.64        36

    accuracy                           0.61        66
   macro avg       0.60      0.60      0.60        66
weighted avg       0.61      0.61      0.61        66



# xgboost

In [258]:
import xgboost as xgb

In [259]:
dtrain = xgb.DMatrix(X_train_std, label=y_train)
param = {'max_depth': 5, 'eta': 1, 'objective': 'multi:softmax', 'num_class': 8} 
num_round = 100 
bst = xgb.train(param, dtrain, num_round)  

In [260]:
dtest = xgb.DMatrix(X_test_std)  
pred = bst.predict(dtest) 
score = accuracy_score(y_test, pred)
print('score:{0:.4f}'.format(score))  

score:0.6515
